In [1]:
from lxml import html

# from bs4 import BeautifulSoup
from bs4 import BeautifulSoup, NavigableString
import time
import datetime
import threading
from apscheduler.schedulers.background import BackgroundScheduler

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

import json
import os
import sys

import helper

In [2]:
course_schedule = []
NO_WINDOW = False
IMPLICIT_WAIT_TIME = 20
EXPLICIT_WAIT_TIME = 20
WINDOW_WIDTH = 720
WINDOW_HEIGHT = 920

In [3]:
# def sync_get_element_by_xpath(xpath):
#     wait.until(ec.visibility_of_element_located((By.XPATH, xpath)))
#     return driver.find_element_by_xpath(xpath)

In [4]:
def create_new_driver():
    #setup option for chrome profile
    # chrome_options = Options()
    # chrome_options.add_argument("user-data-dir=/Users/huohsien/Library/Application Support/Google/Chrome/Default/")
    options = Options()
    
    if NO_WINDOW:
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-gpu')

    #start web driver
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    driver.implicitly_wait(IMPLICIT_WAIT_TIME)

    wait = WebDriverWait(driver, EXPLICIT_WAIT_TIME)
    return driver, wait

In [5]:
def sign_in(driver, wait):
    
    
    username = driver.find_element_by_id('ctl00_cphContents_txtUsername')
    password = driver.find_element_by_id('ctl00_cphContents_txtPassword')
    username.send_keys("DN20092360")
    password.send_keys("jj1216")

    wait.until(ec.visibility_of_element_located((By.CLASS_NAME, "submit-button")))
    
    sign_in_btn = driver.find_element_by_class_name("submit-button") 
    sign_in_btn.click()      
        

def log_out(driver, wait):
    click_btn = helper.sync_get_element_by_xpath(driver, "//a[contains(text(), 'Logout')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()
    
def click_book_for_class(driver, wait):
    click_btn = helper.sync_get_element_by_xpath(driver, "//a[contains(text(), 'Book for Class')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()

def click_book_this_class_now(driver, wait):
    click_btn = helper.sync_get_element_by_xpath(driver, "//a[contains(text(), 'BOOK THIS CLASS NOW')]")
    click_btn.click()

In [6]:
#launch url
base_url = "https://www.trueclassbooking.com.tw/member/search-class.aspx"

In [7]:
with open(r'/Users/huohsien/workspace/python/yoga_class_smart_booking/class_schedules/21 September - 27 September 2020.json', "r") as read_file:
    course_schedule = json.load(read_file)

In [8]:
# course_schedule

In [9]:
# book_datetime = datetime.datetime.now() + datetime.timedelta(seconds=10)

# courses_to_be_booked = []

# c = helper.search_courses_by_date('17', course_schedule)
# # c = helper.search_courses_by_name('Vinyasa Yoga', c)
# c = helper.search_courses_by_teacher('Mei', c)

# courses_to_be_booked.extend(c)

# helper.list_courses_to_be_booked(courses_to_be_booked)

In [10]:
book_datetime = datetime.datetime(2020, 9, 23, 22, 0, 0)
courses_to_be_booked = []
temp = []

c = helper.search_courses_by_date('27', course_schedule)
# c = helper.search_courses_by_name('', c)
c = helper.search_courses_by_teacher('YaYa', c)
c = helper.search_courses_by_time('PM', c)
# c = helper.search_courses_by_name('Hatha', c)
# courses_to_be_booked.extend(c)

temp.extend(c)
courses_to_be_booked.append(temp[1])
courses_to_be_booked.append(temp[2])
    
helper.list_courses_to_be_booked(courses_to_be_booked)


Date: 27 Sep Sun
-----------------------

Name: Universal Yoga 1 寰宇瑜珈 1
Teacher: YaYa
Time: 02:00 - 03:30 PM


Name: Yin Yoga 陰瑜珈
Teacher: YaYa
Time: 03:45 - 04:45 PM




In [11]:
signin_datetime = book_datetime - datetime.timedelta(seconds=600)

In [12]:
# signin_datetime = book_datetime - datetime.timedelta(seconds=6)

In [13]:
def scheduled_sign_in(driver, wait):
#     print("scheduled_sign_in({},{})".format(driver, wait))
    driver.get(base_url)
    sign_in(driver, wait)

    
def reserve_class(driver, wait, course_to_be_booked):
    print("reserve_class({},{},{})".format(driver, wait, course_to_be_booked['time']))
    book_url = course_to_be_booked['href']
    driver.get(book_url)
    click_book_this_class_now(driver, wait)

In [14]:
drivers = []
waits = []
scheduler = BackgroundScheduler()
start = time.time()
for idx, course_to_be_booked in enumerate(courses_to_be_booked):

    driver, wait = create_new_driver()
    driver.set_window_position(WINDOW_WIDTH * idx, 0);
    driver.set_window_size(WINDOW_WIDTH, WINDOW_HEIGHT)
    signin_datetime = signin_datetime - datetime.timedelta(seconds=(idx+1))
#     print("driver: ", driver, " datetime: ", signin_datetime)
    scheduler.add_job(scheduled_sign_in, 'date', run_date=signin_datetime, args=[driver, wait])
    scheduler.add_job(reserve_class, 'date', run_date= book_datetime, args=[driver, wait, course_to_be_booked])
# print("Time Elapsed: {:.2f}".format(time.time() - start)) 
scheduler.start()
 


[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/huohsien/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache


[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/huohsien/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache
